In [1]:
##libraries required
import math
import re
from nltk.metrics import ConfusionMatrix
import random
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy
import csv
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import pandas as pd
import seaborn
from nltk.metrics import ConfusionMatrix
import nltk
from nltk.stem import WordNetLemmatizer 

# Read DAta

In [2]:
# Read the target values from joineddata
pdfJoinedFinal = pd.read_csv('joineddata.csv')
YData = pdfJoinedFinal.MAIN_PRODUCT.values

In [3]:
len(YData)

383066

In [4]:
# Read the features from finalfeatures
pdfFeatures = pd.read_csv('finalfeatures.csv')

In [5]:
cols= pdfFeatures.columns

In [6]:
finalCols = cols.delete([0])

In [7]:
finalCols

Index(['WAS_USER_DISPUTED', 'DATE_COMPLAINT_USER', 'COMPANY',
       'COMPANY_RESPONSE_TO_USER', 'COMPANY_RESPONSE_TO_PUBLIC',
       'WAS_RESPONSE_TIMELY', 'DATE_COMPLAINT_COMPANY', 'MAIN_ISSUE',
       'SUB_ISSUE', 'DIFF_DATE',
       ...
       '990', '991', '992', '993', '994', '995', '996', '997', '998', '999'],
      dtype='object', length=1013)

In [8]:
pdfFeatures.columns

Index(['Unnamed: 0', 'WAS_USER_DISPUTED', 'DATE_COMPLAINT_USER', 'COMPANY',
       'COMPANY_RESPONSE_TO_USER', 'COMPANY_RESPONSE_TO_PUBLIC',
       'WAS_RESPONSE_TIMELY', 'DATE_COMPLAINT_COMPANY', 'MAIN_ISSUE',
       'SUB_ISSUE',
       ...
       '990', '991', '992', '993', '994', '995', '996', '997', '998', '999'],
      dtype='object', length=1014)

In [9]:
pdfFeatures = pdfFeatures[finalCols]

In [10]:
XData = np.asarray(pdfFeatures)

In [11]:
len(XData)

383066

In [12]:
len(XData[0])

1013

# MODEL TRAINING

In [16]:
XData = XData[:100]
YData = YData[:100]

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(XData, YData, test_size=0.2, random_state=42, shuffle=True)

In [18]:
# Use Dict Vectorizer for other text content as the number of uniques are limited . 
# We have already extracted the tf idf vector for complaint text part
print('Creating Vectorizer for whole data set')
vectorizer = DictVectorizer(sparse=True)

vectorizer_input=[]

for eachRow in X_train:
    columnNumber = 0
    temp_dic={}
    for data in eachRow:
        temp_dic[str(columnNumber)] = data
        columnNumber = columnNumber +1
    vectorizer_input.append(temp_dic)
print("Dictonary complete")
output_all = vectorizer.fit_transform(vectorizer_input)
with open('dictvectorizer.pkl', 'wb') as fid:
    pickle.dump(vectorizer, fid)

print("Saving vectorizer model")

Creating Vectorizer for whole data set
Dictonary complete
Saving vectorizer model


In [19]:

print('Creating MAX ABS SCALER for whole data set')
maxabsscaler = MaxAbsScaler()
output_maxabs=maxabsscaler.fit_transform(output_all)
with open('maxscaler.pkl', 'wb') as fid:
    pickle.dump(maxabsscaler, fid)

print("Saving MAX ABS SCALER model")


Creating MAX ABS SCALER for whole data set
Saving MAX ABS SCALER model


In [20]:
# Use Vectorizer and scaler
output = vectorizer.transform(vectorizer_input)

output_transformed = maxabsscaler.transform(output.toarray())

In [21]:
# MultiLayer Perceptron
from sklearn.neural_network import MLPClassifier 
clf = MLPClassifier(solver='sgd', alpha=1e-5,
                 hidden_layer_sizes=(100,50), verbose=1, max_iter=10)
clf.fit(output_transformed, Y_train) 



Iteration 1, loss = 0.67441954
Iteration 2, loss = 0.67059685
Iteration 3, loss = 0.66519900
Iteration 4, loss = 0.65845029
Iteration 5, loss = 0.65053082
Iteration 6, loss = 0.64160978
Iteration 7, loss = 0.63185820
Iteration 8, loss = 0.62147024
Iteration 9, loss = 0.61058513
Iteration 10, loss = 0.59931241


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(100, 50), max_iter=10,
              solver='sgd', verbose=1)

In [22]:
with open('trainedmodel.pkl', 'wb') as fid:
    pickle.dump(clf, fid)


# MODEL EVALUATION

In [23]:

def evaluate_prediction(prediction,test_sample): #for batch testing
    tp=0
    if str(prediction) == str(test_sample):
        tp=1
    return tp


In [24]:
import time
startTime = time.time()
corr=0
total=0
predictionvalue = []
vectorizerlist=[]
for j in range(0, 1):
    columnNumber = 0
    temp_vectorizer=[]
    temp_dic={}
    for data in X_test[j]:
        temp_dic[str(columnNumber)] = data
        columnNumber = columnNumber +1
    temp_vectorizer.append(temp_dic)
    vectorizerlist.append(temp_vectorizer)
    
    test_raw_transformed = vectorizer.transform(temp_vectorizer)

    print(test_raw_transformed)    
    test_raw_scaled = maxabsscaler.transform(test_raw_transformed)

    
    #print(test_raw_scaled.shape)
    #test_raw_scaled_sc = sc.transform(test_raw_transformed.toarray())
    #test_raw_scaled_robust = robustsc.transform(test_raw_transformed.toarray())
    
    prediction = clf.predict(test_raw_scaled.toarray()).tolist()
    predictionvalue.append(prediction[0])
    #print("Prediction : ",prediction[0])
    #print(" Actual: ",test_tag[j])
    cor=evaluate_prediction(prediction[0],Y_test[j])
    corr=corr+cor
    #if cor ==1 and str(test_tag[j]) != 'Normal':
        #print(test_raw[j])
        #print(test_tag[j])
    total=total+1
print(str(corr))
print(str(total))
print('Accuracy is : ' + str(float(corr)/total))
endTime= time.time()
print("Time Taken "+ str(endTime - startTime) )

  (0, 0)	0.0
  (0, 1)	2019.0
  (0, 2)	0.0
  (0, 3)	0.0
  (0, 4)	0.0
  (0, 5)	0.0
  (0, 6)	0.0
  (0, 7)	0.0
  (0, 8)	0.0
  (0, 9)	0.0
  (0, 10)	0.0
  (0, 11)	0.07849939799028242
  (0, 12)	0.0
  (0, 13)	0.0
  (0, 14)	0.0
  (0, 15)	0.0
  (0, 16)	0.0
  (0, 17)	0.0
  (0, 18)	0.0
  (0, 19)	0.0
  (0, 20)	0.0
  (0, 21)	0.0
  (0, 22)	0.0
  (0, 23)	0.0
  (0, 24)	0.0
  :	:
  (0, 1037)	0.0
  (0, 1038)	0.0
  (0, 1039)	0.0
  (0, 1040)	0.0
  (0, 1041)	0.19104693907333115
  (0, 1042)	0.0
  (0, 1043)	0.0
  (0, 1044)	0.0
  (0, 1045)	0.0
  (0, 1046)	0.0
  (0, 1047)	0.0
  (0, 1048)	0.0
  (0, 1049)	0.0
  (0, 1050)	0.0
  (0, 1051)	0.0
  (0, 1052)	0.0
  (0, 1053)	0.0
  (0, 1054)	0.0
  (0, 1055)	0.0
  (0, 1056)	0.0
  (0, 1057)	0.0
  (0, 1058)	0.0
  (0, 1059)	0.0
  (0, 1060)	0.0
  (0, 1061)	0.0
1
1
Accuracy is : 1.0
Time Taken 0.0047228336334228516


In [25]:
import pandas
from pandas_ml import ConfusionMatrix
import matplotlib.pyplot as plt
cm = ConfusionMatrix(y_true=Y_test, y_pred=predictionvalue)
cm.print_stats()

AttributeError: module 'sklearn.metrics' has no attribute 'jaccard_similarity_score'